# Image Classification wiht Transfer Learning 

<table class="tfo-notebook-buttons" align="left">

  <td>
    <a target="_blank" href="https://drive.google.com/file/d/1YNAzDri4S6H6KnltgurK4VFhEFsyWDWC/view?usp=sharing"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab or Download from Google Drive</a>
  </td>


</table>

In this tutorial, you will learn how to classify images of building rooves by using transfer learning from a pre-trained network.


## 1. Data preparation

### 1.1 Data download

In [1]:
# We'll need wget for downloading images in this example. 
# So, install wget firstly.
!pip install wget

You should consider upgrading via the '/Users/simcenter/Codes/Venvs/venv3.7.0_TF2/bin/python -m pip install --upgrade pip' command.


In [2]:
# Import packages 
import wget
import zipfile
import shutil

# Download images prepared by SimCenter
wget.download("https://zenodo.org/record/3986721/files/Roof_Satellite_Images.zip", 'Roof_Satellite_Images.zip')

# unzip images to a local folder
with zipfile.ZipFile('Roof_Satellite_Images.zip', 'r') as zip_ref:
        zip_ref.extractall('.')
        shutil.rmtree('__MACOSX')



In [3]:
# There images are 1000 images in each sub-folder under Roof_Satellite_Images
!ls Roof_Satellite_Images

flat   gabled hipped


### 1.2 Split data

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.inception_v3 import preprocess_input


ImportError: cannot import name 'image_dataset_from_directory' from 'tensorflow.keras.preprocessing' (/Users/simcenter/Codes/Venvs/venv3.7.0_TF2/lib/python3.7/site-packages/tensorflow_core/python/keras/api/_v2/keras/preprocessing/__init__.py)

In [ ]:
img_dir = 'Roof_Satellite_Images'
image_size = (256, 256)
batch_size = 32

print('* First split the data with 8:2.')
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    img_dir,
    validation_split=0.2,
    subset="training",
    seed=1993,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical'
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    img_dir,
    validation_split=0.2,
    subset="validation",
    seed=1993,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical'
)

print("* Then take 20% out of the validation for final testing.")
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches // 5)
val_ds = val_ds.skip(val_batches // 5)

class_names = train_ds.class_names
print('The names of the classes are: ',class_names)


# Configure the dataset for performance 
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

## 2. Create model 


### 2.1 Load the pre-trained base model

In [ ]:
# Load InceptionV3 model pre-trained on imagenet
base_model = tf.keras.applications.InceptionV3(input_shape=image_size + (3,),
                                               include_top=False,
                                               weights='imagenet')
# Freeze the base model
base_model.trainable = False

### 2.2 Add preprocessing layers and a classification head to build the model
(Chaining together the data preprocessing, base_model and feature extractor layers using the [Keras Functional API](https://www.tensorflow.org/guide/keras/functional)). 

In [ ]:
# Pre-processing layer
inputs = tf.keras.Input(shape=image_size + (3,))
x = preprocess_input(inputs) 

# Then go into the backbone model
x = base_model(x)

# Then go into the classification header
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.6)(x) # You can change the dropout rate 
prediction_layer = tf.keras.layers.Dense(len(class_names), activation='softmax')
outputs = prediction_layer(x)

# Put them together
model = tf.keras.Model(inputs, outputs)



### 2.3 Compile the model



In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

## 3. Train the model




In [ ]:
initial_epochs = 30
history = model.fit(train_ds, epochs=initial_epochs, validation_data=val_ds)

In [ ]:
# Plot learning curves

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## 4. Fine tuning


### 4.1 Un-freeze the top layers of the model


In [ ]:
# Un-freeze the whole base model
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 300 # There are a total of 311 layer

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False


### 4.2 Compile the model


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate/10),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
#model.summary()

### 4.3 Continue training the model

In [ ]:
fine_tune_epochs = 30
total_epochs =  initial_epochs + fine_tune_epochs
history_fine = model.fit(train_ds, epochs=total_epochs, initial_epoch=history.epoch[-1], validation_data=val_ds)


In [ ]:
# Plot learning curves

acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## 5. Evaluate the performance of the model

In [ ]:
# Evaluate the overall performance on the test set
loss, accuracy = model.evaluate(test_ds)
print('Test accuracy :', accuracy)

In [ ]:
# Save the model in the current folder 
model.save('roof_classifier.h5') 

# Download the trained model to your computer
from google.colab import files
files.download('roof_classifier.h5')

## 6. Use the model for prediction

In [ ]:
import random
import numpy as np
from glob import glob
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model


model = load_model('roof_classifier.h5')
class_names = ['flat', 'gabled', 'hipped']
imgList = glob('Roof_Satellite_Images/*/*.png')

random.shuffle(imgList)

plt.figure(figsize=(10, 10))
for i in range(9):
  img_path = imgList[i]
  ax = plt.subplot(3, 3, i + 1)
  img = image.load_img(img_path, target_size=(256, 256))
  x = image.img_to_array(img)
  plt.imshow(x.astype("uint8"))
  
  x = np.expand_dims(x, axis=0)

  prediction = model.predict(x)
  prediction = np.argmax(prediction[0])
  plt.title(class_names[prediction])
  plt.axis("off")